In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt

In [2]:
def sigmoid(z: np.array):
    s = 1 / (1+np.exp(-z))
    return s

In [3]:
def layer_sizes(X, Y):
    n_x = X.shape[0]
    n_y = Y.shape[0]

    return n_x, n_y 

In [4]:
def intialize_paramters(n_x, n_h, n_y):
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y,1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [5]:
def forward_propagation(X, parameters):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)

    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [6]:
def compute_cost(A2, Y):
    m = Y.shape[1]

    logprobs = np.multiply(Y, np.log(A2)) + np.multiply((1-Y), np.log(1-A2))
    cost = - np.sum(logprobs) / m

    cost = float(np.squeeze(cost))

    return cost

In [7]:
def backward_propagation(parameters, cache, X, Y):
    m = X.shape[1]

    W1 = parameters['W1']
    W2 = parameters['W2']

    A1 = cache['A1']
    A2 = cache['A2']

    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m
    dZ1 = np.multiply(np.dot(W2.T, dZ2), (1- np.power(A1, 2)))
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m

    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads



In [8]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']

    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters


In [9]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[1]

    parameters = intialize_paramters(n_x, n_h, n_y)

    for i in range(num_iterations):
        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y)
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, grads)


        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters


In [10]:
def predict(parameters, X):
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 > 0.5).astype(int)

    return predictions

In [11]:
np.random.seed(1)
X = np.random.randn(2, 100)  # 2 features and 100 samples
Y = (np.sum(X, axis=0) > 0).reshape(1, 100)  # Labels: 1 if sum of features > 0, otherwise 0

# Running the neural network
n_h = 4  # Number of neurons in the hidden layer
parameters = nn_model(X, Y, n_h, num_iterations=10000, print_cost=True)

# Making predictions
predictions = predict(parameters, X)
print("Predictions:", predictions)


Cost after iteration 0: 0.693122
Cost after iteration 1000: 0.006703
Cost after iteration 2000: 0.003775
Cost after iteration 3000: 0.002569
Cost after iteration 4000: 0.001912
Cost after iteration 5000: 0.001502
Cost after iteration 6000: 0.001224
Cost after iteration 7000: 0.001025
Cost after iteration 8000: 0.000876
Cost after iteration 9000: 0.000761
Predictions: [[1 1 0 0 0 0 1 0 1 0 1 0 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0
  0 0 1 1 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0 1
  1 1 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1]]
